## 가상의 문서를 이용한 하이드
하이드 (HyDE, Hypothetical Document Embedding) 는 검색 성능을 향상시키는 방법 중 하나입니다. 전
통적인 검색 시스템에서는 사용자의 질의를 직접 임베딩하여 문서를 검색했지만, 하이드는 질의에 대한
가상의 답변 문서 (LLM 의 답변을 하이드에서는 가상의 문서라고 명명합니다.) 를 먼저 생성한 후 이를 기
반으로 검색을 수행하여 검색 성능을 높이는 방법입니다. 실제 구현을 통해서 하이드 방법론에 대해서 이해해봅시다

###1. 하이드(HyDE, Hypothetical Document Embedding)의 동작 과정
하이드의 동작 과정은 세단계로 나눌 수 있습니다.

1. 가상 문서 생성 단계: 이 단계는 LLM 을 활용하여 사용자의 질문에 대한 답변 문서를 생성하는 단계입
니다. 예를 들어 사용자의 질문에 답변해주는 하이드 기반의 의료 시스템이 있다고 가정해봅시다. 사용자
가 “코로나 19 의 주요 증상은 무엇인가요?” 라는 질문을 했다고 해봅시다. 기존의 RAG 에서는 사용자의
해당 질문으로 벡터 데이터베이스에서 유사한 문서를 검색하는 경우가 많았습니다. 하지만 하이드에서
는 해당 질문을 바로 검색으로 사용하지 않고 준비해둔 LLM 에게 질문을 입력합니다.

- LLM 의 답변: “코로나 19 의 주요 증상으로는 발열 (37.5 도 이상), 기침, 호흡곤란, 오한, 근육통, 두
통, 인후통, 후각·미각소실 등이 있습니다. 증상은 감염 후 평균 5‑7 일 이내에 나타나며, 경증에서
중증까지 다양한 범위로 발현될 수 있습니다.”

LLM 은 위와 같이 일반적인 답변을 할 것입니다. 이 답변에는 할루시네이션이 있을 수도 있지만 이 답변
은 최종 답변이 아니므로 괜찮습니다.

2. 임베딩 및 검색 단계: 위에서 얻은 LLM 의 답변을 임베딩 모델을 통해 벡터화하고, 이 벡터를 사용하여
벡터 데이터베이스에서 유사한 문서들을 검색합니다. 다시 말해 사용자의 질문으로 검색하는 것이 아니
라 사용자의 질문에 대한 LLM 의 일반적인 답변으로 검색을 수행하는 것입니다. 예를 들어 다음과 같은
문서들이 검색될 수 있습니다.

• 문서 1 (유사도: 0.92): “코로나 19 임상적 특징: 주요 증상으로는 발열 (38 도 이상), 마른기침, 피로
감이 있으며…”

• 문서 2 (유사도: 0.85): “WHO 코로나 19 증상 가이드라인: 가장 흔한 증상은 발열, 기침, 피로감입니
다…”

• 문서 3 (유사도: 0.76): “코로나 19 감염자 통계 분석: 증상별 발현 비율을 보면 발열 89%, 기침
68%…”

3.  최종 답변 생성: 벡터 데이터베이스에서 검색된 관련 문서들을 컨텍스트로 활용하여 LLM 이 최종 답변
을 생성합니다. 이때 검색된 문서가 사용자의 질문으로 검색한 것보다 LLM 의 답변으로 검색했을 때 검색
성능이 향상되었다고 한다면 더 좋은 답변을 얻을 수 있을 것입니다.

llm질문 -> llm답변 -> llm답변 임베딩 -> 벡터 데이터베이스 질문(llm 답변 임베딩) -> llm 질문(벡터 데이터베이스 답변)

###2. 하이드의 장점과 주의 해야 할 점
사용자의 질문을 임베딩하여 검색하는 것이 아니라, 사용자의 질문에 대해서 LLM 의 답변을 얻고 LLM 답
변으로 검색하는 방법, 하이드의 장점에는 어떤 것이 있을까요?

**첫째, 질의의 의미적 확장성**입니다. 질문을 직접 검색하는 대신 LLM 의 답변 문서로 검색하면 기존 사용
자 질의의 의미를 자연스럽게 확장하여 검색을 수행할 수 있습니다. “감기 예방법” 이라는 간단한 질문에
대해서도, 가상 문서는 손 씻기, 마스크 착용, 충분한 수면 등 구체적인 방법들을 포함하게 되어 더 풍부한
검색이 가능해집니다. 또 다른 예를 들어봅시다. “주식 투자 초보자 팁” 이라는 질문에 대해, 가상 문서는
분산 투자의 중요성, 장기 투자 전략, 리스크 관리 등 초보자에게 필요한 포괄적인 정보를 포함할 수 있습
니다.

**둘째, 검색 정확도 향상**입니다. 실제 벡터 데이터베이스 검색에서는 놓칠 수 있는 관련 문서들을 가상 문서를 통해 더 잘 찾아낼 수 있습니다. 의미적으로 유사하지만 다른 표현을 사용한 문서들도 효과적으로 검색됩니다. 이는 실제 다양한 연구를 통해 검색 성능을 측정하는 다양한 메트릭에 대해서 정량적으로 증명된 사항입니다.

하이드 구현 시 고려해야 할 주요 요소들도 있습니다.
먼저 **가상 문서의 품질**이 중요합니다. LLM 이 생성하는 가상 문서의 품질이 전체 시스템의 성능을 좌우하므로, 적절한 LLM 의 선택과 프롬프트 엔지니어링을 통해 고품질의 가상 문서가 생성되도록 해야 합니다.

**연산 비용**도 중요한 고려사항입니다. LLM 을 사용하여 가상 문서를 생성하는 과정이 추가되므로 기존 검색 방식대비 LLM 호출이 1 회 추가 발생합니다. 실시간 검색이 필요한 경우 이를 고려하여 시스템을 설계를 해야 합니다.

도메인에 적절한지 고려해야 하는 도메인 특화 문제도 있습니다. 법률, 의료, 금융 등 특정 도메인에서는
가상 문서 생성을 위한 프롬프트를 해당 도메인의 특성에 맞게 조정해야 할 수 있습니다. 예를 들어 법률
문서 검색과 의료 문서 검색에서는 각각 다른 스타일의 가상 문서 (LLM 의 답변) 가 필요할 수 있습니다.
이러한 특성들을 고려하여 하이드를 구현한다고 하였을 때 더욱 정확하고 관련성 높은 검색 결과를 제공
할 수 있습니다. 이제 실제 구현을 통해서 하이드를 이해해봅시다.

###3. 하이드 구현하기
**1) 패키지 설치 및 OpenAI키 값 설정**
랭체인 패키지와 벡터 데이터베이스를 위한 langchain_chroma, PDF 를 읽기 위한 pypdf 를
설치합니다.

In [1]:
!pip install langchain_openai langchain_community langchain_chroma pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.2/313.2 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 73.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 105.8 MB/s eta 0:00

In [2]:
import os
import urllib.request
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma

OpenAi의 키값을 셋팅합니다.

In [3]:
os.environ['OPENAI_API_KEY'] = 'Openai_api_key'

**2) 데이터 다운로드**
사용할 데이터는 2023_ 북한인권보고서.pdf 입니다.

In [4]:
# 분석할 PDF 파일을 웹에서 다운로드.
urllib.request.urlretrieve("https://github.com/llama-index-tutorial/llama-index-tutorial/raw/main/ch07/2023_%EB%B6%81%ED%95%9C%EC%9D%B8%EA%B6%8C%EB%B3%B4%EA%B3%A0%EC%84%9C.pdf",
                           filename="2023_북한인권보고서.pdf")

('2023_북한인권보고서.pdf', <http.client.HTTPMessage at 0x797e4afaf010>)

**3) 거대 언어 모델과임베딩 설정**
LangChain을 사용하여 실습에서 사용할 각종 설정들듸 값을 정합니다.


In [5]:
# LangChain의 LLM과 임베딩 모델 설정
llm = ChatOpenAI(model="gpt-4o", temperature = 0.2)
embed_model = OpenAIEmbeddings(model="text-embedding-3-large")

# 문서 분할 설정
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size =300,
    chunk_overlap = 100,
)
# PDF 문서를 읽고 벡터 인덱스 생성
loader = PyPDFLoader("2023_북한인권보고서.pdf")
documents = loader.load()
chunks = text_splitter.split_documents(documents)
vector_store = Chroma.from_documents(chunks, embed_model)

**4) 하이드 구현하기**

In [20]:
def generate_hypothetical_doc(question: str) -> str:
  """질문에 대한 가상의 이상적인 답변 문서 생성"""
  prompt = f"""주어진 질문에 대해, 마치 실제 문서에서 발췌한 것 같은 이상적인 답변을 작성해주세요.
  단, 구체적인 수치, 날짜, 트랜드와 같은 상세 정보를 포함해야 합니다.

  질문: {question}

  답변:"""

  response = llm.invoke(prompt)
  return response.content

generate_hypothetical_doc 함수는 하이드의 첫 번째 단계인 가상 문서 생성을 담당합니다. 이
함수는 사용자의 질문을 입력받아 검색없이 거대 언어 모델이 임의의 답변을 생성합니다.
함수에서는 가상 문서 생성을 위한 프롬프트가 prompt란 변수에 작성되어져 있습니다. LLM 에게 실제
문서처럼 보이는 답변을 생성하도록 지시합니다. 특히’ 구체적인 수치, 날짜, 트렌드와 같은 상세 정보를 포함해야 합니다’ 라는 지시를 통해 답변을 생성합니다.

생성된 프롬프트는 llm.invoke를 통해 gpt-4o로 전달되어 이에 대한 응답을 생성합니다. 함수는 이
응답의 텍스트 부분을 최종적으로 반환합니다. 이렇게 생성된 가상 문서는 이후 실제 문서 검색을 위한 쿼리로 사용됩니다.


In [25]:
def search_with_hyde(hypothetical_doc: str):
  """가상 문서를 이용해 실제 문서 검색"""
  results = vector_store.similarity_search_with_score(hypothetical_doc, k=4)
  return [
      {
          'content': doc.page_content,
          'score': score
      } for doc, score in results
  ]

search_with_hyde 함수는 하이드의 두번째 단계인 검색 과정을 수행 합니다. 앞서 generate_hypothetical_doc에서 생성한 가상 문서를 입력으로 받아, 이를 기반 으로 실제 문서를 검색합니다.

함수는vector_store.similarity_search_with_score를 호출하여 가상 문서와 가장 유사한 문서들을 검색합니다. 이 메서드는 k=4 파라미터를 통해 상위 4 개의 문서를 반환하며, 각 문서와 함께 유사도 점수도 함께 반환합니다.검색 결과는 각 검색된 문서 doc에 대해 문서의 내용 content와 유사도 점수 score를 딕셔너리 형태로 구성합니다. doc.page_content는 문서의 실제 텍스트 내용을 가져오고, score는 해당 문서와 가상 문서 간의 유사도 점수를 나타냅니다. 이렇게 가공된 검색 결과는 최종 답변 생성을 위한 다음 단계로 전달되게 됩니다.


In [22]:
def generate_final_answer(question: str, relevant_docs: list) -> str:
  """검색된 문서를 바탕으로 최종 답변 생성"""
  context = '\n\n'.join([doc['content'] for doc in relevant_docs])

  prompt = f"""다음 검색 결과를 바탕으로 질문에 답변해주세요.
  검색 결과의 정보를 최대한 사용하고, 없는 정보는 답변하지 마세요.

  검색결과:
  {context}

  질문: {question}

  답변:"""

  response = llm.invoke(prompt)
  return response.content

generate_final_answer 함수는 하이드의 마지막 단계인 최종 답변 생성을 담당하는 함수입니다.
이 함수에서 가장 중요한 부분은 context 변수를 만드는 과정입니다.

context = "\n\n".join([doc['content'] for doc in relevant_docs]) 코드를 보면, search_with_hyde 함수에서 찾아온 4 개의 검색 결과가 하나의 문자열로 연결되어 context 변수에 저장됩니다. 이렇게 저장된 context는 프롬프트의 {context} 부분에 삽입됩니다.

그 다음으로 prompt라는 변수에 프롬프트를 구성하는데, 이 프롬프트는 LLM 에게 검색된 문서들의 정보만을 사용하여 답변을 생성하도록 명확히 지시합니다. ’ 검색 결과의 정보를 최대한 사용하고, 없는 정보는 답변하지 마세요’ 라는 제약 조건을 통해 LLM 이 검색된 문서의 범위를 벗어나지 않도록 제한합니다. 이는 LLM 이 자체적으로 정보를 생성하거나 추측하는 것을 방지하고, 오직 검색된 문서의 내용만을 기반으로 답변하도록 하기 위함입니다. 프롬프트의 {question} 부분에는 사용자가 처음에 물어본 질문이 그대로 들어갑니다.

최종적으로 llm.invoke를 통해 이 prompt 변수에 저장된 프롬프트를 gpt-4o에 전달하고, 생
성된 답변의 텍스트를 반환합니다. 여기서 llm은 전역 설정에서 정의된 gpt-4o 모델을 의미하며,
invoke()는 이 모델에게 프롬프트를 전달하고 응답을 받아오는 역할을 합니다. 이렇게 생성된 답변은
검색된 문서들의 정보에 기반한 신뢰할 수 있는 응답이 됩니다. 즉, 모든 답변의 내용은 검색된 문서들 안에서 찾을 수 있는 정보들로만 구성되어 있어 사실 관계를 신뢰할 수 있습니다.


In [23]:
def process_query(question: str):
  """전체 HyDE 프로세스"""
  # 1. 가상 문서 생성
  print("1. 가상문서 생성 중.....")
  hypothetical_doc = generate_hypothetical_doc(question)
  print("\n가상 문서:", hypothetical_doc)

  # 2. 가상 문서로 검색
  print("\n2. 실제 문서 검색 중.....")
  relevant_docs = search_with_hyde(hypothetical_doc)

  # 3. 최종 답변 생성
  print("\n3. 최종 답변 생성 중.....")
  final_answer = generate_final_answer(question, relevant_docs)

  return {
      "hypothtical_doc":hypothetical_doc,
      "retrieved_docs": relevant_docs,
      "final_answer" : final_answer
  }

process_query 함수는 하이드의 전체 프로세스를 실행하는 메인 함수입니다. 사용자의 질문을 입력
받아 가상 문서 생성, 문서 검색, 최종 답변 생성이라는 세 단계를 순차적으로 실행합니다.

첫 번째 단계에서는 generate_hypothetical_doc 함수에 question을 전달하여 가상 문서를 생
성하고 이를 hypothetical_doc 변수에 저장합니다. 이 과정에서 "1. 가상 문서 생성 중..."이
라는 진행 메시지와 함께 생성된 가상 문서의 내용을 출력합니다.

두 번째 단계에서는 search_with_hyde 함수에 hypothetical_doc을 전달하여 실제 문서 검색
을 수행하고, 그 결과를 relevant_docs 변수에 저장합니다. "2. 실제 문서 검색 중..."이라는
진행 메시지가 출력됩니다.

세 번째 단계에서는 generate_final_answer 함수에 원래 질문 question과 검색된 문서들
relevant_docs을 전달하여 최종 답변을 생성하고 이를 final_answer 변수에 저장합니다."3.
최종 답변 생성 중..."이라는 진행 메시지가 출력됩니다.

마지막으로 함수는 생성된 가상 문서 hypothetical_doc, 검색된 문서들 retrieved_docs, 그리고 최종 답변final_answer을 딕셔너리 형태로 묶어서 반환합니다. 이 반환값을 통해 전체 프로세스
의 각 단계 결과를 확인할 수 있습니다.
아래 코드는 process_query 함수를 실제로 실행하고 그 결과를 순차적으로 출력하는 부분입니다.

In [26]:
question = "북한에서 강제로 이루어지는 조직 생활은 무엇이 있나요?"
result = process_query(question)

print("\n==== 프로세스 결과 ====")
print("\n[가상 문서]")
print(result["hypothtical_doc"])
print("\n[검색된 문서들]")
for idx, doc in enumerate(result["retrieved_docs"],1):
  print(f"\n문서 {idx} (유사도 점수: {doc['score']:.4f}):")
  print(doc['content'])
print("\n[최종 답변]")
print(result["final_answer"])

1. 가상문서 생성 중.....

가상 문서: 북한에서 강제로 이루어지는 조직 생활은 주로 주민들의 사상 교육과 사회 통제를 목적으로 운영됩니다. 대표적인 예로는 '인민반'과 '청년동맹'이 있습니다.

1. **인민반**: 인민반은 북한 주민들이 속해 있는 가장 기초적인 사회 조직 단위로, 보통 20~40가구로 구성됩니다. 인민반은 주민들의 일상 생활을 감시하고 통제하는 역할을 하며, 정기적인 회의와 교육 세션을 통해 주민들에게 당의 정책과 지침을 전달합니다. 인민반 회의는 주로 매주 열리며, 주민들은 이 회의에 반드시 참석해야 합니다. 회의에서는 주민들의 생활 상황을 점검하고, 사상 교육을 실시하며, 당의 지시 사항을 전달합니다.

2. **청년동맹**: 조선사회주의노동청년동맹, 줄여서 청년동맹은 북한의 청소년과 젊은이들이 의무적으로 가입해야 하는 조직입니다. 청년동맹은 14세부터 30세까지의 청소년과 청년을 대상으로 하며, 이들은 정기적인 모임과 교육을 통해 김일성-김정일주의를 학습하고, 사회주의 건설에 기여할 것을 요구받습니다. 청년동맹 활동은 주로 사상 교육, 노동 동원, 체육 행사 등으로 구성되며, 이를 통해 젊은 세대의 충성심을 강화하고 사회주의 체제에 대한 지지를 공고히 합니다.

이 외에도 북한 주민들은 다양한 형태의 조직 생활에 참여해야 하며, 이러한 활동은 주민들의 개인 생활에 큰 영향을 미칩니다. 이러한 조직 생활은 주민들의 일상적인 삶을 통제하고, 체제에 대한 충성심을 강화하는 수단으로 활용되고 있습니다.

2. 실제 문서 검색 중.....

3. 최종 답변 생성 중.....

==== 프로세스 결과 ====

[가상 문서]
북한에서 강제로 이루어지는 조직 생활은 주로 주민들의 사상 교육과 사회 통제를 목적으로 운영됩니다. 대표적인 예로는 '인민반'과 '청년동맹'이 있습니다.

1. **인민반**: 인민반은 북한 주민들이 속해 있는 가장 기초적인 사회 조직 단위로, 보통 20~40가구로 구성됩니다. 인민반은 주민들의 일상 생활을 감시하고 

위의 실행결과에서 나타난 유사도 점수들을 보면, 점수가 높을 수록 거리가 멀다는 의미 입니다. 따라서 문서1이 가장 유사하고, 문서4가 상대적으로 덜 유사한 것을 알 수 있습니다.

지금까지 리랭킹과 하이드라는 두 가지 방식을 통해 검색 시스템의 성능을 향상시키는 방법을 살펴보았
습니다. 리랭킹은 초기 검색 결과를 더욱 정교하게 재정렬하여 사용자의 의도에 가장 부합하는 문서를 선
별해내는 방식이며, 하이드는 가상의 이상적인 답변을 생성하여 이를 기반으로 검색의 정확도를 높이는 방식입니다. RAG 시스템에 이러한 검색 성능 향상 기법들을 필요에 따라 적절히 적용한다면 검색의 정확도를 향상시키고 더욱 신뢰할 수 있는 답변을 얻을 수 있습니다.


**6) 코사인 거리**
위의 실행 결과에서 검색 문서의 유사도도 같이 출력하였습니다. 크로마 DB 는 코사인 유사도를 직접 사용하는 대신 코사인 거리 (cosine distance) 개념을 사용합니다. 코사인 유사도 (cosinesimilarity) 는 1 에 가까울수록 두 벡터가 유사하다는 것을 의미합니다 (완전히 같은 방향이면 1)